In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv


The code below defines the path for the dataset and makes sure it's on Kaggle.
I ran this notebook on Kaggle since it's much easier to access the data there and work with it.

In [2]:
from pathlib import Path
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/us-patent-phrase-to-phrase-matching')
    ! pip install -q datasets

In [3]:
import pandas as pd
df = pd.read_csv(path/'train.csv')
df.head(5)

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [4]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


Here we create an input feature for the model.
We need to tell the model somehow about where a certain feature starts and where it ends.
So, We use Text1 and Text2 and ANC1, It's arbitrary tho.

In [5]:
df['input'] = 'Text1: ' + df.context + '; Text2: ' + df.target + '; ANC1: ' + df.anchor

In [6]:
df.input

0        Text1: A47; Text2: abatement of pollution; ANC...
1        Text1: A47; Text2: act of abating; ANC1: abate...
2        Text1: A47; Text2: active catalyst; ANC1: abat...
3        Text1: A47; Text2: eliminating process; ANC1: ...
4        Text1: A47; Text2: forest region; ANC1: abatement
                               ...                        
36468    Text1: B44; Text2: wooden article; ANC1: wood ...
36469    Text1: B44; Text2: wooden box; ANC1: wood article
36470    Text1: B44; Text2: wooden handle; ANC1: wood a...
36471    Text1: B44; Text2: wooden material; ANC1: wood...
36472    Text1: B44; Text2: wooden substrate; ANC1: woo...
Name: input, Length: 36473, dtype: object

# Tokenization

Transformers need a dataset object to work with. Here we create one.

In [7]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [8]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

We use transfer learning and an already pre-trained model to build our model on top of that.
deberta is a good model to start with for this particular dataset (and possibly, many other nlp tasks in general).
Then we use AutoTokenizer from the transformers library to instantiate out tokenizer.

In [9]:
model_nm = 'microsoft/deberta-v3-small'
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


to tokenize the whole dataset we use a map function to map a function to all the data that we have. This will return a new ds object called 'tok_ds'. tok_ds has few more features as well, like input_ids which includes the id for each token.

In [10]:
def tok_fn(x): return tokz(x['input'])
tok_ds = ds.map(tok_fn, batched = True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [11]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

For some reason, transformer library needs the column to be named 'labels' so here we rename it to that.

In [12]:
tok_ds = tok_ds.rename_columns({'score': 'labels'})

# Test and Validation sets

In [13]:
test_df = pd.read_csv(path/'test.csv')
test_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


Transformers contain a train test split method to split the dataset into a dataset dict object, so we use it here.

In [14]:
dds = tok_ds.train_test_split(0.25, seed = 5)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

Creating an input feature for our validation test set

In [15]:
test_df['input'] = 'TEXT1: ' + test_df.context + '; TEXT2: ' + test_df.target + '; ANC1: ' + test_df.anchor
test_ds = Dataset.from_pandas(test_df).map(tok_fn, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

The metric for this competition is "Pearson correlation coefficient" based on Kaggle information. So, here we create one.

In [16]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}
def corr(x,y): return np.corrcoef(x,y)[0][1]

# Training the model

In [17]:
from transformers import TrainingArguments,Trainer
bs = 128
epochs = 4
lr = 8e-5

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


The code below is the default settings for transformers. Transformers use TrainingArguments class to set up the arguments for the training.

In [18]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [20]:
trainer.train();

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.028527,0.804022
2,No log,0.021342,0.827778
3,0.037900,0.021496,0.835894
4,0.037900,0.022486,0.836230


In [21]:
preds = trainer.predict(test_ds).predictions.astype(float)
preds

array([[ 0.47607422],
       [ 0.7890625 ],
       [ 0.49462891],
       [ 0.24841309],
       [-0.03820801],
       [ 0.50585938],
       [ 0.5078125 ],
       [-0.0094223 ],
       [ 0.29101562],
       [ 1.09179688],
       [ 0.24157715],
       [ 0.27368164],
       [ 0.7890625 ],
       [ 0.76464844],
       [ 0.74609375],
       [ 0.47851562],
       [ 0.31396484],
       [-0.01925659],
       [ 0.68945312],
       [ 0.37817383],
       [ 0.43920898],
       [ 0.26269531],
       [ 0.27514648],
       [ 0.2220459 ],
       [ 0.58007812],
       [-0.01332092],
       [-0.01815796],
       [-0.0221405 ],
       [-0.02296448],
       [ 0.75927734],
       [ 0.17834473],
       [-0.01089478],
       [ 0.68457031],
       [ 0.48535156],
       [ 0.41455078],
       [ 0.19946289]])

In [22]:
preds = np.clip(preds, 0, 1)
preds

array([[0.47607422],
       [0.7890625 ],
       [0.49462891],
       [0.24841309],
       [0.        ],
       [0.50585938],
       [0.5078125 ],
       [0.        ],
       [0.29101562],
       [1.        ],
       [0.24157715],
       [0.27368164],
       [0.7890625 ],
       [0.76464844],
       [0.74609375],
       [0.47851562],
       [0.31396484],
       [0.        ],
       [0.68945312],
       [0.37817383],
       [0.43920898],
       [0.26269531],
       [0.27514648],
       [0.2220459 ],
       [0.58007812],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.75927734],
       [0.17834473],
       [0.        ],
       [0.68457031],
       [0.48535156],
       [0.41455078],
       [0.19946289]])

In [23]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': test_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1013